# Exercise: Creating the Bob In-Flight HTLC Commitment Transaction

Now as exercise let's create the Bob In-Flight Commitment Transaction.

In [ ]:
%run "../chapter 2 - initial commitment transaction/initial commitment transaction.ipynb"

## The Unsigned Transaction

### The Input

In [ ]:
# VERSION
# version '2' indicates that we may use relative timelocks (BIP68)
version = bytes.fromhex("0200 0000")

# MARKER (new to segwit)
marker = bytes.fromhex("00")

# FLAG (new to segwit)
flag = bytes.fromhex("01")

# INPUTS
# We have just 1 input
input_count = bytes.fromhex("01")

# Convert txid and index to bytes (little endian)
txid = (bytes.fromhex(funding_channel_txid))[::-1]
funding_channel_index = 0
index = funding_channel_index.to_bytes(4, byteorder="little", signed=False)

# For the unsigned transaction we use an empty scriptSig
scriptsig = bytes.fromhex("")

# sequence: upper 8 bits are 0x80, lower 24 bits are the upper 24 bits of the obscured commitment number
# Commitment number of the second commitment transaction.
commitment_number = 1
# obscured commitment number is result of xor operation 
commitment_number_obscured = lower48_to_obscure ^ commitment_number
# Extract the upper 24 bits of the obscured commitment number
upper_24_bits = (commitment_number_obscured >> 24) & 0xFFFFFF
# Combine the upper 8 bits (0x80) with the lower 24 bits (upper 24 of obscured number)
sequence = (0x80 << 24) | upper_24_bits
# Convert to bytes (byte, big-endian)
sequence = sequence.to_bytes(4, byteorder='big')
# Convert to little-endian 
sequence = sequence[::-1]

inputs = (
    txid
    + index
    + varint_len(scriptsig)
    + scriptsig
    + sequence
)

print("Commitment Number Osbscured", hex(commitment_number_obscured))

### The Outputs

Bob transaction will have 4 outputs also:
* local anchor output
* remote anchor output
* accepted htlc output
* to remote output

#### The Accepted HTLC Output

    +------+---------------+
    | OP_1 |       Q       |
    +------+---------------+
                   ^
                   |   +-------------------+
                    ---| P(revocation) + T |
                       +-------------------+
                                         ^
                                         |
                                   +-----------+        
                                   | T = t * G |
                                   +-----------+        
                                         ^
                                         |
     +---+   +-------------------------------------------------------+
     | t | = | TaggedHash ("Taptweak", P(revocation) || script_root) |
     +---+   +-------------------------------------------------------+
                                                             ^
                                                             |
                                                          +-----+
                          ------------------------------> | hAB |<------------------
                         |                                +-----+                   |
                         |                                                          |
                         |                                                          |
                      +----+                                                     +----+
                      | hA |                                                     | hB |
                      +----+                                                     +----+
                         ^                                                          ^
                         |                                                          |
      +----------------------------------------------------+                        |
      | P(remote_htlc) OP_CHECKSIG OP_1 OP_CSV OP_DROP     |                        |
      |                                                    |                        |
      | cltv_expiry OP_CHECKLOCKTIMEVERIFY OP_DROP         |                        |
      +----------------------------------------------------+                        |
      +-------------------------------------------------------------------------------+ 
      | OP_SIZE 32 OP_EQUALVERIFY OP_HASH160 <RIPEMD160(payment_hash)> OP_EQUALVERIFY |
      |                                                                               |
      | P(local_htlc) OP_CHECKSIGVERIFY P(remote_htlc) OP_CHECKSIG                    |
      +-------------------------------------------------------------------------------+

In [ ]:
# TODO: Create Alice per-commitment using per_commitment function with alice_per_commitment_seed and commitment_number
alice_per_commitment = # TODO
# TODO: Create Bob per-commitment using per_commitment function with bob_per_commitment_seed and commitment_number
bob_per_commitment = # TODO

# TODO: Create Alice Local Public Key (P(remote) for Bob)
# Use the method 'get_pubkey' from the basepoint object (alice_payment) and the alice per commitment point
alice_payment_pubkey = # TODO
print(f"Alice Payment PubKey: {alice_payment_pubkey.get_bytes(bip340=True).hex()}")

# TODO: Create Bob Delayed Public Key using derivate_key with bob_node_seed, family=4, channel_index=0
bob_delayed_pubkey = # TODO
print(f"Bob Delayed PubKey: {bob_delayed_pubkey.get_bytes(bip340=True).hex()}")

# TODO: Create Alice Revocation Public Key using derivate_revocation_key
alice_revocation_pubkey = # TODO
print(f"Alice Revocation PubKey: {alice_revocation_pubkey.get_bytes(bip340=True).hex()}")

# TODO: Create Alice HTLC Public Key using derivate_key with family=2
alice_htlc_pubkey = # TODO
print(f"Alice HTLC PubKey: {alice_htlc_pubkey.get_bytes(bip340=True).hex()}")

# TODO: Create Bob HTLC Public Key using derivate_key with family=2
bob_htlc_pubkey = # TODO
print(f"Bob HTLC PubKey: {bob_htlc_pubkey.get_bytes(bip340=True).hex()}")

# Outputs for Bob In-Flight Commitment Transaction
# 0x04 outputs
output_count = bytes.fromhex("04")

# ANCHOR OUTPUTS
# Anchor output amount (we use the same amount as before)

# OP_16 OP_CSV
script = CScript([OP_16, OP_CHECKSEQUENCEVERIFY])

# Method: ser_string(data) is a function which adds compactsize to input data.
hash_input = TAPSCRIPT_VER + ser_string(script)

# Anchor Output script_root
script_root = tagged_hash("TapLeaf", hash_input)

# TODO: # Create the taptweak using Alice Payment Pubkey Tagged Hash and script_root
taptweak = # TODO
alice_payment_pubkey_tweaked = alice_payment_pubkey.tweak_add(taptweak)
# scriptPubKey P2TR: OP_1 (0x51) + PUSH32 (0x20) + xonly(32B)
alice_anchor_spk = bytes.fromhex("51") + varint_len(alice_payment_pubkey_tweaked.get_bytes(bip340=True)) + alice_payment_pubkey_tweaked.get_bytes(bip340=True)

# TODO: Bob Anchor Output Tagged Hash (local anchor for Bob)
# Use tagged_hash with "TapTweak" and bob_delayed_pubkey + script_root
taptweak = # TODO
bob_delayed_pubkey_tweaked = bob_delayed_pubkey.tweak_add(taptweak)
# scriptPubKey P2TR: OP_1 (0x51) + PUSH32 (0x20) + xonly(32B)
bob_anchor_spk = bytes.fromhex("51") + varint_len(bob_delayed_pubkey_tweaked.get_bytes(bip340=True)) + bob_delayed_pubkey_tweaked.get_bytes(bip340=True)

# Sort by scriptPubKey (lexicographic order)
anchors = [
    (alice_anchor_spk, "alice_anchor"),
    (bob_anchor_spk,   "bob_anchor"),
]

anchors_sorted = sorted(anchors, key=lambda x: x[0])

# ACCEPTED HTLC OUTPUT
# HTLC output amount
htlc_output_value_satoshis = 500000
htlc_output_value = htlc_output_value_satoshis.to_bytes(8, byteorder="little", signed=False)

# TODO: Create the leaf scripts A and B
# script A: P(remote_htlc) OP_CHECKSIG OP_1 OP_CSV OP_DROP cltv_expiry OP_CHECKLOCKTIMEVERIFY OP_DROP
# Add cltv_delta (40 blocks) to current block height
cltv_expiry = node.getblockcount() + 40
scriptA = # TODO

# We use a dummy payment_hash here. In a real scenario, this would be the hash of the preimage for the HTLC.
# And would be sent in the message update_add_htlc.
payment_preimage = sha256(b'payment_secret').digest()
sha = sha256(payment_preimage).digest()
payment_hash = hashlib.new("ripemd160", sha).digest()

# TODO: Leaf script B: OP_SIZE 32 OP_EQUALVERIFY OP_HASH160 <RIPEMD160(payment_hash)> OP_EQUALVERIFY P(local_htlc) OP_CHECKSIGVERIFY P(remote_htlc) OP_CHECKSIG
scriptB = # TODO

# TODO: Compute TapLeaves A and B
# Method: ser_string(data) is a function which adds compactsize to input data.
hash_inputA = # TODO
hash_inputB = # TODO

# TODO: Use the tagged_hash function to compute TapLeaf A and B
htlc_taggedhash_leafA = # TODO
taggedhash_leafB = # TODO

# Method: Returns tapbranch hash. Child hashes are lexographically sorted and then concatenated.
# l: tagged hash of left child
# r: tagged hash of right child
def tapbranch_hash(l, r):
    return tagged_hash("TapBranch", b''.join(sorted([l,r])))

# TODO: Compute Internal node TapBranch AB
root_ab = # TODO

# TODO: Compute TapTweak for alice_revocation_pubkey
taptweak = # TODO

# TODO: Tweak the alice revocation pubkey
alice_revocation_pubkey_tweaked = # TODO

# scriptPubKey P2TR: OP_1 (0x51) + PUSH32 (0x20) + xonly(32B)
bob_accepted_htlc_spk = bytes.fromhex("51") + varint_len(alice_revocation_pubkey_tweaked.get_bytes(bip340=True)) + alice_revocation_pubkey_tweaked.get_bytes(bip340=True)

# Second commitment expected weight 
commitment_weight = taproot_commit_weight(NumTapOut=2, NumAnchorOut=2)
print("Commitment Transaction Expected Weight:", commitment_weight)

# From open_channel and update_feerate messages (1 sat/vb = 250 sat/kw)
feerate_per_kw = 250

# Calculate the fee for the second commitment transaction
commitment_fee = int(commitment_weight * feerate_per_kw / 1000)

# TO REMOTE OUTPUT
# to remote output amount
# As we have no to_local output in this case, the to_remote value pay the fees
# Using the same fee as before
to_remote_value_sat =  channel_value_sat - anchor_output_value_satoshis * 2 - htlc_output_value_satoshis - commitment_fee
to_remote_value = to_remote_value_sat.to_bytes(8, byteorder="little", signed=False)
print("channel_value_sat, anchor_output_value_satoshis * 2, commitment_fee: ", channel_value_sat, anchor_output_value_satoshis, commitment_fee)

# TODO: P(remote) OP_CHECKSIG OP_1 OP_CSV OP_DROP
# Use CScript to create the to remote script
script = # TODO

# Method: ser_string(data) is a function which adds compactsize to input data.
hash_input = TAPSCRIPT_VER + ser_string(script)

# TODO: To Remote Output script_root
# Use the tagged_hash function to compute the script root
script_root = # TODO

# TODO: To Remote Output Tagged Hash
# Use the tagged_hash function to compute the taptweak for NUMS
taptweak = # TODO
# Use the tweak_add to tweak the NUMS key
NUMS_tweaked = NUMS.tweak_add(taptweak)
# Create the scriptPubKey P2TR: OP_1 (0x51) + PUSH32 (0x20) + xonly(32B)
bob_to_remote_spk = bytes.fromhex("51") + varint_len(NUMS_tweaked.get_bytes(bip340=True)) + NUMS_tweaked.get_bytes(bip340=True)

outputs = (
    anchor_output_value
    + varint_len(anchors_sorted[0][0])
    + anchors_sorted[0][0]
    + anchor_output_value
    + varint_len(anchors_sorted[1][0])
    + anchors_sorted[1][0]
    + htlc_output_value
    + varint_len(bob_accepted_htlc_spk)
    + bob_accepted_htlc_spk
    + to_remote_value
    + varint_len(bob_to_remote_spk)
    + bob_to_remote_spk
)

# Locktime is the same

unsigned_tx = (
    version
    + input_count
    + inputs
    + output_count
    + outputs
    + locktime
)
print("unsigned_tx: ", unsigned_tx.hex())

# Decode the unsigned transaction to verify it looks correct
decoded = node.decoderawtransaction(unsigned_tx.hex())
print(json.dumps(decoded, indent=2, default=str))

## The sighash for the key path spend

In [ ]:
index_of_this_input = bytes.fromhex("0000 0000")

# SIGHASH for key path spend
# See BIP-341 for details
sighash_epoch = bytes.fromhex("00")

# Control
hash_type = bytes.fromhex("00") # SIGHASH_DEFAULT (a new sighash type meaning implied SIGHASH_ALL)

# Transaction data
sha_prevouts = sha256(txid + index).digest()

sha_amounts = sha256(channel_value).digest()

# scriptPubKey P2TR: OP_1 (0x51) + PUSH32 (0x20) + xonly(32B)
sha_scriptpubkeys = sha256(
    varint_len(channel_spk)
    + channel_spk
).digest()

sha_sequences = sha256(sequence).digest()

sha_outputs = sha256(outputs).digest()

# Data about this input
spend_type = bytes.fromhex("00") # no annex present

sig_msg = (
    sighash_epoch
    + hash_type
    + version
    + locktime
    + sha_prevouts
    + sha_amounts
    + sha_scriptpubkeys
    + sha_sequences
    + sha_outputs
    + spend_type
    + index_of_this_input
)

tag_hash = sha256("TapSighash".encode()).digest()
sighash = sha256(tag_hash + tag_hash + sig_msg).digest()

## Signing the sighash

In [ ]:
# Reusing the reordered participants to match the sorted pubkeys

# Alice and Bob generates its own nonce pair (secnonce, pubnonce)
secnonces, pubnonces = [], []
for pk, sk, pcs in participants:
    # Use per-commitment nonce for bob to get deterministic results and nonce_gen for Alice jit nonce
    if pk == pubkey_bob_musig2.get_bytes(bip340=False):
        sec, pub = nonce_per_commitment(pcs, commitment_number, sk, pk, agg_pubkey_tweaked, sighash)
    else:
        sec, pub = nonce_gen(sk, pk, agg_pubkey_tweaked, sighash, None)
    secnonces.append(sec)
    pubnonces.append(pub)

# TODO: Alice and Bob has exchanged already the pubnonces, so they can aggregate them and create the session context
# Use nonce_agg to aggregate the pubnonces
agg_nonce = # TODO
# TODO: Use the SessionContext to create the signature session
session = # TODO

# TODO: Alice and Bob produces their partial signatures
# Use sign function to produce the partial signatures
psigs = # TODO

# Each side verify the other's partial signature before proceeding
for i, psig in enumerate(psigs):
    assert partial_sig_verify(psig, pubnonces, sorted_pubkeys, [bip86_tweak], [True], sighash, i)

# TODO: Each side combines partial signatures into the final Schnorr signature
# Use the function partial_sig_agg to aggregate the partial signatures
agg_sig = # TODO

# Sanity check: verify with BIP340 against the *tweaked* x-only key ---
ok = schnorr_verify(sighash, agg_pubkey_tweaked, agg_sig)
print("aggregated Schnorr verifies?", ok)

## The signed transaction

In [ ]:
witness = (
    bytes.fromhex("01") # one stack item in the witness
    + varint_len(agg_sig)
    + agg_sig
)

# the final signed transaction
signed_bob_commitment_tx = (
    version
    + marker
    + flag
    + input_count
    + inputs
    + output_count
    + outputs
    + witness
    + locktime
)

print("signed tx: ",signed_bob_commitment_tx.hex())
# Decode the unsigned transaction to verify it looks correct
decoded = node.decoderawtransaction(signed_bob_commitment_tx.hex())
print(json.dumps(decoded, indent=2, default=str))

print(node.testmempoolaccept(rawtxs=[signed_bob_commitment_tx.hex()]))